Chess RL + Expert Learning + Weirdness 

In [ ]:
import wandb
import chess
import gym
import chess
import os, sys, copy
import torch
from torch.utils.data import Dataset, DataLoader, random_split, Subset
from torch.cuda.amp import GradScaler
from torch.cuda.amp import autocast
from tqdm import tqdm
import numpy as np
import time
import adversarial_gym
from adversarial_gym.chess_env import ChessEnv

from OBM_ChessNetwork import Chess42069NetworkSimple

sys.path.append('../../chess_utils')
from chess_dataset import ChessDataset
from utils import RunningAverage

Model Gameplay Functions

In [ ]:
def play_game(env, white, black, perspective=None, sample_n=1, duel=False):
    step = 0
    actions = []
    log_probs = []
    observations = []
    values = []
    done_mask = []
    done = False
    obs = env.reset()[0]
    while not done:
        # Note: Modify the get_action function to return the action, log_prob, and state value
        if step % 2 == 0:
            action_logits, value_estimate = white(obs[0])
            action, log_prob = white.to_action(action_logits, env.board.legal_moves, sample_n) # same for black/white
        else:
            action_logits, value_estimate = black(obs[0])
            action, log_prob = black.to_action(action_logits, env.board.legal_moves, sample_n) # same for black/white

        if perspective is None or perspective == chess.WHITE and step % 2 == 0 or perspective == chess.BLACK and step % 2 == 1:
            observations.append(obs[0])
            actions.append(action)
            log_probs.append(log_prob)
            values.append(value_estimate)
            done_mask.append(0 if not done else 1)

        obs, reward, done, _, info = env.step(action)
        step += 1
        
    if reward in [-1,1]:
        print(f"PERSPECTIVE: {perspective} - GAME OUTCOME: {reward}")
    else:
        print(f"DRAW")

    # Reward is 1 if chosen perspective won and -1 if chosen perspective lost
    # If dueling set reward to 1 for a win and zero otherwise
    if perspective is not None:
        reward = 1 if (reward == -1 and perspective == chess.BLACK) else reward
        reward = -1 if (reward == 1 and perspective == chess.BLACK) else reward

        if duel and reward != 1: reward = 0

    rewards = prepare_game_rewards(reward, perspective, len(actions)) if not duel else reward

    return observations, actions, log_probs, values, done_mask, rewards


def prepare_game_rewards(reward, perspective, game_len):
    if perspective is not None:
        # Reward in [-1, 0, 1] 
        rewards = [reward for _ in range(game_len)]
        return rewards
    
    # Self play alternates +/- reward
    # Reward in [-1,0,1].
    rewards = [reward if i % 2 == 0 else -reward for i in range(game_len)]

    rewards = compute_discounted_rewards(reward)
    return rewards
    
    
def compute_discounted_rewards(reward, gamma=0.99):
    """Compute discounted rewards for a sequence of rewards."""
    n = len(reward)
    discounted_rewards = [0] * n
    running_add = 0
    for t in reversed(range(n)):
        running_add = running_add * gamma + reward[t]
        discounted_rewards[t] = running_add
    return discounted_rewards


def duel(env, old_model, new_model, num_rounds):
    """ Duel against the previous best model and return the win ratio. """
    new_model.eval()
    with torch.no_grad():
        wins = 0
        for i in range(num_rounds):
            _, _, _, _, _, r_w = play_game(env, new_model, old_model, perspective=chess.WHITE, sample_n = 2, duel=True)
            _, _, _, _, _, r_b = play_game(env, old_model, new_model, perspective=chess.BLACK, sample_n = 2, duel=True)

            wins += r_w + r_b
    new_model.train()    
    return wins / (2 * num_rounds)


def self_play(env, model, num_games):
    """ Plays num_games against itself to gather obs, actions, log_probs, rewards data """
    # TODO: check if numpy array of shape (num_games, 4) is faster, each row could be output of play_game
    actions = []
    log_probs = []
    rewards = []
    observations = []
    for _ in range(num_games):
        g_obs, g_actions, g_log_probs, g_reward = play_game(env, model, model, perspective=None)
        actions.append(g_actions)
        log_probs.append(g_log_probs)
        rewards.append(g_reward)
        observations.append(g_obs)
    return observations, actions, log_probs, rewards

Expert Learning

In [ ]:
def run_validation(model, val_loader, stats):
    model.eval()
    stats.reset("val_loss")
    t1 = time.perf_counter()
    with torch.no_grad():
        for i, (state, action, result) in enumerate(val_loader):
            state = state.float().to('cuda' if torch.cuda.is_available() else 'cpu')
            action = action.to('cuda' if torch.cuda.is_available() else 'cpu')
            result = result.float().to('cuda' if torch.cuda.is_available() else 'cpu')
            
            policy_output, value_output = model(state.unsqueeze(1))
            policy_loss = model.policy_loss(policy_output.squeeze(), action)
            value_loss = model.val_loss(value_output.squeeze(), result)
            
            loss = policy_loss + value_loss
            stats.update("val_loss", loss.item())
    
    print(f"Mean Validation Loss: {stats.get_average('val_loss')}, time elapsed: {time.perf_counter()-t1} seconds")
    return stats.get_average('val_loss')


def expert_study(model, dataset, percent_dataset=0.1):
    """ Trains on TCEC data in a supervised fashion (behaviour cloning)"""

    # Load random subset of dataset and split
    study_size = int(percent_dataset * len(dataset))
    random_indices = np.random.randint(0, study_size, study_size)
    study_dataset = Subset(dataset, random_indices)
    
    train_ratio = 0.9
    train_size = int(train_ratio * study_size)
    val_size = study_size - train_size
    train_dataset, val_dataset = random_split(study_dataset, [train_size, val_size])

    # Create data loaders for the training and validation sets
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, 
                            pin_memory=False, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, 
                            pin_memory=False, num_workers=2)

    stats = RunningAverage()
    stats.add(["train_loss", "val_loss", "train_p_loss", "train_v_loss"])

    model.train()
    t1 = time.perf_counter()
    for i, (state, action, result) in enumerate(train_loader):
        state = state.float().to(model.device)
        action = action.to(model.device)
        result = result.float().to(model.device)

        with autocast():
            policy_output, value_output = model(state.unsqueeze(1))
            policy_loss = model.policy_loss(policy_output.squeeze(), action)
            value_loss = model.val_loss(value_output.squeeze(), result)
            loss = policy_loss + value_loss
        
        # AMP with gradient clipping
        model.optimizer.zero_grad()
        model.grad_scaler.scale(loss).backward()
        model.grad_scaler.unscale_(model.optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        model.grad_scaler.step(model.optimizer)
        model.grad_scaler.update()

        stats.update({
            "train_loss": loss.item(),
            "train_p_loss": policy_loss.item(),
            "train_v_loss": value_loss.item()
            })
        
    print(f"Study Train Loss: {stats.get_average('train_loss')}")
    # wandb.log({"study_train_loss": stats.get_average('train_loss')})
    t2 = time.perf_counter()
    valid_loss = run_validation(model, val_loader, stats)
    # wandb.log({"val_loss": valid_loss, "iter": i})


Load Models

In [ ]:
# Load Model
MODEL_PATH = '/home/kage/chess_workspace/simpler_SwinChessNet42069.pt'

model = Chess42069NetworkSimple(hidden_dim=256, device='cuda')
best_model = Chess42069NetworkSimple(hidden_dim=256, device='cuda')

if os.path.exists(MODEL_PATH):
    print("Loading model at: {MODEL_PATH}")
    model.load_state_dict(torch.load(MODEL_PATH))
    best_model.load_state_dict(torch.load(MODEL_PATH))

best_model.eval()

Train - VPG with Self-Play and Dueling

In [ ]:
# Train Params
PGN_FILE = '/home/kage/chess_workspace/PGN-data/alphazero_stockfish_all/alphazero_vs_stockfish_all.pgn'
MODEL_SAVEPATH = '/home/kage/chess_workspace/WACKY_RL_MODEL.pt'

NUM_EPOCHS = 100
STUDY_EVERY = 1 
DUEL_EVERY = 10

chess_dataset = ChessDataset(PGN_FILE)
env = gym.make("Chess-v0")

In [ ]:
for i in tqdm(range(NUM_EPOCHS)):
    # Play games as white or black against the previous best model
    if i % 2 == 0:
        observations, actions, log_probs, values, done_mask, rewards = play_game(env, model, best_model, perspective=chess.WHITE, sample_n=3)
    else:
        observations, actions, log_probs, values, done_mask, rewards = play_game(env, best_model, model, perspective=chess.BLACK, sample_n=3)
    
    next_values = values[1:] + [0] # Value of next state, 0 for final action

    # Convert data to PyTorch tensors
    # observations = torch.as_tensor(observations, dtype=torch.float32, device=model.device)
    actions = torch.as_tensor(actions, dtype=torch.int64, device= model.device)
    log_probs = torch.stack(log_probs).to(model.device)
    values = torch.as_tensor(values, dtype=torch.float32, device= model.device)
    rewards = torch.as_tensor(rewards, dtype=torch.float32, device= model.device)
    next_values = torch.as_tensor(next_values, dtype=torch.float32, device= model.device)
    done_mask = torch.as_tensor(done_mask, dtype=torch.float32, device= model.device)

    model.update_network(log_probs, rewards, values, next_values, done_mask, gamma=0.99)

    # # # Expert Study
    # # # if i % STUDY_EVERY == 0:
    # # if True:
    # #     expert_study(model, chess_dataset, percent_dataset=0.05)

    # Darwinian duel to the death
    if i % DUEL_EVERY == 0:
        win_ratio = duel(env, best_model, model, num_rounds=10)
        print(f"Model win ratio: {win_ratio}")
        if win_ratio > 0.6:
            print("Best model was deafeted!")
            best_model = copy.deepcopy(model)
            torch.save(model.state_dict(), MODEL_SAVEPATH)
            best_model.eval()

    # # Self play
    # obss, actions, log_probs, reward = play_game(env, model, model, perspective=None)
    # model.update_policy(model, log_probs, rewards)